In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p1/mnist_test.csv
/kaggle/input/2021-ai-w7-p1/mnist_train.csv
/kaggle/input/2021-ai-w7-p1/submission.csv


In [2]:
#data road

import torch

torch.manual_seed(1);

train = pd.read_csv("../input/2021-ai-w7-p1/mnist_train.csv")
test  = pd.read_csv("../input/2021-ai-w7-p1/mnist_test.csv")
submit = pd.read_csv("../input/2021-ai-w7-p1/submission.csv")

In [3]:
#gpu 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
train

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,784
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
59996,59996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
59997,59997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
59998,59998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [5]:
# 데이터 정리  필요 없는 열 제거
train = train.drop("Unnamed: 0", axis = 1)
test = test.drop("Unnamed: 0", axis = 1)

In [6]:
# 데이터 분류
x_train = train.drop("784", axis = 1)
y_train = train['784']

In [7]:
#standardscaler 를 통한 데이터 정규화
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)
test = sc.transform(test)


In [8]:
#tensor 자료형으로 변환
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
test = torch.FloatTensor(test)

In [9]:
#dataset으로 변환
from torch.utils.data import DataLoader,TensorDataset
train_set = TensorDataset(x_train, y_train)

In [10]:
#학습 파라미터 설정
lr = 0.001
training_epochs =10
batch_size = 100
drop_prob = 0.3

In [11]:
# 모델 만들기
layer1 =  torch.nn.Linear(784, 1568)
layer2 =  torch.nn.Linear(1568, 1568)
layer3 =  torch.nn.Linear(1568, 1568)
layer4 =  torch.nn.Linear(1568, 784)
layer5 =  torch.nn.Linear(784, 10)

relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p = drop_prob)
model  = torch.nn.Sequential(layer1,relu,dropout,
                            layer2,relu,dropout,
                            layer3,relu,dropout,
                            layer4,relu,dropout,
                            layer5).to(device)


In [12]:
#weight 초기화
torch.nn.init.xavier_normal(layer1.weight)
torch.nn.init.xavier_normal(layer2.weight)
torch.nn.init.xavier_normal(layer3.weight)
torch.nn.init.xavier_normal(layer4.weight)
torch.nn.init.xavier_normal(layer5.weight)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  


Parameter containing:
tensor([[ 0.0347,  0.0800, -0.0422,  ...,  0.0485,  0.1148, -0.0198],
        [-0.0616,  0.1390,  0.0116,  ...,  0.0019,  0.0277,  0.0206],
        [-0.0162, -0.0268,  0.0426,  ..., -0.0413, -0.0222, -0.0332],
        ...,
        [ 0.0314, -0.0033,  0.0468,  ...,  0.0147,  0.0300, -0.0610],
        [ 0.0808,  0.0377, -0.0642,  ...,  0.0516, -0.0002,  0.0069],
        [-0.0099, -0.0561, -0.0174,  ..., -0.0557, -0.0207, -0.0584]],
       device='cuda:0', requires_grad=True)

In [13]:
# 손실함수, 최적화 함수
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [14]:
data_loader = DataLoader(dataset = train_set,batch_size = batch_size, shuffle = True, drop_last = True)

In [15]:
#모델 학습
model.train()
total_batch = len(data_loader)

for e in range(training_epochs):
    avg_cost = 0
    
    for x,y in data_loader:
        #mnist 자료형
        x = x.view(-1,784).to(device)
        y = y.to(device)
        
        hypothesis = model(x)
        cost = loss(hypothesis , y)
        
        optimizer.zero_grad()
        
        cost.backward()
        
        optimizer.step()
        
        avg_cost = cost / total_batch
    print("epoch : " , e+1 , "cost : ", '{:.9f}'.format(avg_cost))
    
    
        
        
        

epoch :  1 cost :  0.000545330
epoch :  2 cost :  0.000391236
epoch :  3 cost :  0.000355994
epoch :  4 cost :  0.000121122
epoch :  5 cost :  0.000196496
epoch :  6 cost :  0.000071278
epoch :  7 cost :  0.000343145
epoch :  8 cost :  0.000642143
epoch :  9 cost :  0.000031643
epoch :  10 cost :  0.000090800


In [16]:
# 결과
with torch.no_grad():
    model.eval()
    test = test.to(device)
    predict = model(test)
predict = torch.argmax(predict , dim = 1)
predict

tensor([7, 2, 1,  ..., 4, 5, 6], device='cuda:0')

In [17]:
submit

,id,Label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
9995,9995,1
9996,9996,1
9997,9997,1
9998,9998,1


In [18]:
submit['Label'] = predict.detach().cpu().numpy()

In [19]:
submit.to_csv("submission.csv", index = False)
s = pd.read_csv("submission.csv")
s

,id,Label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
...,...,...
9995,9995,2
9996,9996,3
9997,9997,4
9998,9998,5
